# 掛載G-drive


mount 為 Linux掛載的指令，使用AWS掛載EFS也是使用此指令

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pickle
import pandas as pd 
data =pd.read_csv("/content/drive/MyDrive/python暑假課程/2021 3 13_網站與nlp_csv/df_400_.csv")

#Pandas 撈取資料
請從data當中使用pandas套件取出類別為Backgorund與conclusion的資料各200筆(共400筆資料)

In [55]:
#請於課堂上操作
B = data[data['category']=='BACKGROUND'].sample(100)
C = data[data['category']=='CONCLUSIONS'].sample(100)
all_data = pd.concat([B,C],axis=0)

# 文字前處理
1. 將句子小寫
2. 去除部必要的標點符號(使用正則表達式)
3. 使用wordNet還原字根


In [56]:
import re,nltk
from nltk.corpus import stopwords  
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize,regexp_tokenize
wtlem = WordNetLemmatizer()
ps = PorterStemmer()    
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')                                                   #我於2021/1/27 改成使用wordNet
stop_words = set(stopwords.words('english'))  
sent = "The do 8777776786786 789798 789798789 did does done doing done happier #$%^&*IO(*&^%$#@#$%^ %^&*&^%$%^&*&^%$%^) brown fox jumped jumping over a lazy dog"



print(stop_words)
print("原本的句子",sent)
sent_lower = sent.lower() 
sent_stayWord = re.sub(r'[^\w\s]','',sent_lower)                #去除符號 數字資料及本身已經刪除 
print("去除標點符號:",sent_stayWord)
sent_subNumber = re.sub(r'[0-9]','',sent_stayWord)                #去除符號 數字資料及本身已經刪除 
print("去除數字 : ", sent_subNumber)
sent_subNumber.split(" ")
corpus_tokenize = word_tokenize(sent_subNumber)
print("斷詞後" ,corpus_tokenize )
filtered_sentence = [w for w in corpus_tokenize if not w in stop_words]  #刪除停用詞


print(filtered_sentence)
after_preprocess_sent = ""
for i in range(len(filtered_sentence)):                                             
  filtered_sentence[i] = wtlem.lemmatize(filtered_sentence[i],'v')        #wordnet
  after_preprocess_sent = after_preprocess_sent+filtered_sentence[i]+" "
  if(i==(len(filtered_sentence)-1)):
    after_preprocess_sent = after_preprocess_sent+filtered_sentence[i]
  #filtered_sentence[c] = ps.stem(filtered_sentence[c]) 

print(after_preprocess_sent)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
{'about', "needn't", 'wasn', 'through', 'not', "hasn't", "don't", 'didn', 'him', 'you', "didn't", 'out', 'are', 'did', 'will', 'before', "couldn't", 'theirs', 'for', 'should', 'he', 'these', 'doesn', "should've", 'wouldn', 'does', 'very', "isn't", 'has', 'needn', 'their', 'i', "you're", 'then', "haven't", 'there', 'on', 'further', 'both', 'whom', "won't", "mustn't", 'it', 'won', 'itself', 'have', 'here', 've', 'doing', 'me', 'some', 'in', "you'll", 'we', 'yourselves', 'aren', 'her', 't', 're', "wouldn't", 'now', 'my', 'than', 's', 'd', 'only', 'couldn', 'what', 'if', 'own', 'll', 'such', "you've", 'shan', "shan't", 'hers', 'nor', "doesn't", 'and',

# 將寫好的預處理句子寫成函式-方便日後使用(記得寫註解不然會忘記 ex：我今天又重寫了一次)

In [57]:

def preprocess_sent(new_sent):
  new_sent = new_sent.lower() 
  sent_stayWord = re.sub(r'[^\w\s]','',new_sent)                #去除符號 數字資料及本身已經刪除 
  sent_subNumber = re.sub(r'[0-9]','',sent_stayWord)                #去除符號 數字資料及本身已經刪除 
  corpus_tokenize = regexp_tokenize(sent_subNumber, pattern='\w+|\$[\d\.]+|\S+')
  filtered_sentence = [w for w in corpus_tokenize if not w in stop_words]  #刪除停用詞
  after_preprocess_sent = ""
  for i in range(len(filtered_sentence)):                                             
    filtered_sentence[i] = wtlem.lemmatize(filtered_sentence[i],'v')        #wordnet
    after_preprocess_sent = after_preprocess_sent+filtered_sentence[i]+" "
    if(i==(len(filtered_sentence)-1)):
      after_preprocess_sent = after_preprocess_sent+filtered_sentence[i]
  return after_preprocess_sent

preprocess_all_data = [preprocess_sent(x) for x in list(all_data.sentence)]
preprocess_all_data

['interactive mobile health rehabilitation imhere mobile health mhealth system develop support selfmanagement individuals disabilities disabilities',
 'unfortunately quality care plan communication clinicians critically ill patients families situations highly variable frequently lead overutilization invasive icu treatments treatments',
 'webbased interventions focus behavior change use pain management study webbased interventions integrate clinical practice lack lack',
 'potential technologies effectively support health care transformation widely accept accept',
 'accord us national institute health office disease prevention need improve public awareness syndrome among health care providers public public',
 'mobile apps help empower patients health care providers proactively address medical condition near realtime monitor treatment regardless location patient health care provider provider',
 'decline uptake routine childhood vaccinations result outbreaks vaccinepreventable diseases dis

## 以上code 很麻煩對不對? 我也這麼覺得 所以我建議你用Spacy
仔細看會發現第6句就做完所有的事情了，了不起頂多使用nltk的斷句套件

In [58]:
#use spacy for preprocessing
import spacy , re
nlp = spacy.load("en_core_web_sm")
text = "The rain in #$% Spain falls mainly on the plain." 
original_corpus = re.sub(r'[^\w\s]','',text.lower())   #刪除部必要的標點符號 
doc = nlp(original_corpus) 
token = [token.lemma_ for token in doc if token.is_stop == False and token.text.isalpha() == True] #關鍵的第7句
#filtered = [token.text for token in doc if token.is_stop == False and token.text.isalpha() == True]
my_lst_str = ' '.join(map(str, token))
my_lst_str


'''
所以預處理只要以下幾行就好了
import spacy
def preprocess(sentence):
  original_corpus = re.sub(r'[^\w\s]','',sentence.lower())   #刪除部必要的標點符號 
  doc = nlp(original_corpus) 
  token = [token.lemma_ for token in doc if token.is_stop == False and token.text.isalpha() == True] #關鍵的第7句
  my_lst_str = ' '.join(map(str, token))
  retrun my_lst_str

'''

"\n所以預處理只要以下幾行就好了\nimport spacy\ndef preprocess(sentence):\n  original_corpus = re.sub(r'[^\\w\\s]','',sentence.lower())   #刪除部必要的標點符號 \n  doc = nlp(original_corpus) \n  token = [token.lemma_ for token in doc if token.is_stop == False and token.text.isalpha() == True] #關鍵的第7句\n  my_lst_str = ' '.join(map(str, token))\n  retrun my_lst_str\n\n"

# N-gram 使用scikit-learn套件完成

該套件中的CountVectorize為Bag-of-word加上n-gram


In [59]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
     "I like nlnlpiyean xiao."
 ]
vectorizer = CountVectorizer(ngram_range=(1,3))
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
#print()
print(pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names()))

#use our own data

vectorizer = CountVectorizer(ngram_range=(1,3))
X = vectorizer.fit_transform(preprocess_all_data)
print(vectorizer.get_feature_names())
#print()
print(pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names()))


['and', 'and this', 'and this is', 'document', 'document is', 'document is the', 'first', 'first document', 'is', 'is the', 'is the first', 'is the second', 'is the third', 'is this', 'is this the', 'like', 'like nlnlpiyean', 'like nlnlpiyean xiao', 'nlnlpiyean', 'nlnlpiyean xiao', 'one', 'second', 'second document', 'the', 'the first', 'the first document', 'the second', 'the second document', 'the third', 'the third one', 'third', 'third one', 'this', 'this document', 'this document is', 'this is', 'this is the', 'this the', 'this the first', 'xiao']
   and  and this  and this is  ...  this the  this the first  xiao
0    0         0            0  ...         0               0     0
1    0         0            0  ...         0               0     0
2    1         1            1  ...         0               0     0
3    0         0            0  ...         1               1     0
4    0         0            0  ...         0               0     1

[5 rows x 40 columns]
['abilities', 'a

# n-gram進階使用情境
1. 只跟對各類別常出現的單詞做n-gram
2. 根據出現超過n次的單詞才丟入句子做考慮

# TF-IDF 使用scikit-learn套件完成


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2),stop_words=["I","is","nlnlpiyean","xiao"] )
X = vectorizer.fit_transform(preprocess_all_data)
#print(vectorizer.get_feature_names())
print(pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names()))

     abilities  abilities end  ...  youthoptimized  youthoptimized app
0          0.0            0.0  ...             0.0                 0.0
1          0.0            0.0  ...             0.0                 0.0
2          0.0            0.0  ...             0.0                 0.0
3          0.0            0.0  ...             0.0                 0.0
4          0.0            0.0  ...             0.0                 0.0
..         ...            ...  ...             ...                 ...
195        0.0            0.0  ...             0.0                 0.0
196        0.0            0.0  ...             0.0                 0.0
197        0.0            0.0  ...             0.0                 0.0
198        0.0            0.0  ...             0.0                 0.0
199        0.0            0.0  ...             0.0                 0.0

[200 rows x 3891 columns]


# 讓該資料轉成Dataframe

In [61]:
corpus_df = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
print(corpus_df)

     abilities  abilities end  ...  youthoptimized  youthoptimized app
0          0.0            0.0  ...             0.0                 0.0
1          0.0            0.0  ...             0.0                 0.0
2          0.0            0.0  ...             0.0                 0.0
3          0.0            0.0  ...             0.0                 0.0
4          0.0            0.0  ...             0.0                 0.0
..         ...            ...  ...             ...                 ...
195        0.0            0.0  ...             0.0                 0.0
196        0.0            0.0  ...             0.0                 0.0
197        0.0            0.0  ...             0.0                 0.0
198        0.0            0.0  ...             0.0                 0.0
199        0.0            0.0  ...             0.0                 0.0

[200 rows x 3891 columns]


# 請將一開始分類好的文本分別使用n-gram , tf-idf並使用邏輯回歸分類


In [62]:

X = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
print(all_data)
y = all_data.category
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


                                              sentence     category
48   The Interactive Mobile Health and Rehabilitati...   BACKGROUND
70   Unfortunately, the quality of care planning an...   BACKGROUND
71    Web-based interventions with a focus on behav...   BACKGROUND
107  The potential of these technologies to effecti...   BACKGROUND
134  According to the US National Institutes of Hea...   BACKGROUND
..                                                 ...          ...
359   WelTel may be an effective tool for improving...  CONCLUSIONS
286  We will use these findings to iteratively impr...  CONCLUSIONS
234  TRIAL REGISTRATION: Clinicaltrials.gov NCT0043...  CONCLUSIONS
214  TRIAL REGISTRATION: ClinicalTrials.gov NCT0231...  CONCLUSIONS
208   The results show that the PVT mobile cognitiv...  CONCLUSIONS

[200 rows x 2 columns]


## 那我們這邊可使用CountVectorizer+邏輯回歸 分類試試看

題外話: 所有sklearn的svm rf 還是啥軌的 方式都一樣 看看文件就會了

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,1))
X = vectorizer.fit_transform(all_data["sentence"])
print(vectorizer.get_feature_names())
X = (pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names()))

from sklearn.model_selection import train_test_split
y = all_data["category"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
from sklearn.linear_model import LogisticRegression
new_dataset = LogisticRegression(random_state=0).fit(X_train, y_train)
predict_dataet = new_dataset.predict(X_test)

from sklearn.metrics import classification_report
target_names = ['BACKGROUND', 'CONCLUSIONS']
print(classification_report(y_test, predict_dataet, target_names=target_names))

['10', '10095', '10926', '14860', '15', '19', '20', '2011', '2019', '2196', '41', '49', '5615', '5836', '6aa8x3mw1', '6r4p3bub0', '6v0tto0lz', '6wspbuq1m', '6xzryxeif', '72vczmzab', '80', '9166', 'a1c', 'abilities', 'ability', 'about', 'absence', 'abstraction', 'abundant', 'academic', 'accelerometers', 'acceptability', 'acceptable', 'acceptance', 'accepted', 'access', 'accessibility', 'accessible', 'accidental', 'accommodative', 'according', 'accordingly', 'accounts', 'accumulating', 'accuracy', 'accurate', 'achieving', 'acquired', 'across', 'act', 'action', 'actions', 'active', 'actively', 'activities', 'activity', 'add', 'addition', 'additional', 'additionally', 'address', 'addressed', 'adds', 'adequately', 'adherence', 'adjunct', 'adjusting', 'admin', 'administration', 'adolescence', 'adolescents', 'adoption', 'adult', 'adulthood', 'adults', 'advances', 'advantages', 'advertising', 'affect', 'affecting', 'affects', 'africa', 'african', 'after', 'against', 'age', 'agency', 'agents', 

如果你會使用Torch會對Bert的理解非常快 當然Torch也有套件可以做斷詞

In [64]:
import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")
tokens = tokenizer("You can now install TorchText using pip!")
tokens
['you', 'can', 'now', 'install', 'torchtext', 'using', 'pip', '!']

['you', 'can', 'now', 'install', 'torchtext', 'using', 'pip', '!']

In [65]:
type(tokens)

list